In [10]:
import json
import sample_generator
import math
import numpy as np
import random

  
# Opening JSON file
f = open('samples.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
f.close()
  
print(len(data))

10000


In [11]:
# we check the data is somehow well distributed
sample_generator.test(data)

{0.0: 1305, 3.0: 1244, 5.0: 1263, 6.0: 1223, 9.0: 1251, 10.0: 1227, 12.0: 1281, 15.0: 1206}


In [12]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()
    
  def __gt__(self, other):
    return self.data > other.data

In [13]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [22]:
# We define the Neural Network structure
# 4 inputs, first argument
# A pair a hidden layers made by 8 neurons
# 8 outputs (8 possible results)
"""
n = MLP(3, [4, 4, 1])

xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)
    
ypred



"""
n = MLP(4, [4, 4, 8, 4])
xs = [example['input'] for example in data[:5]]
ys = [example['answer'] for example in data[:5]]

In [29]:
for k in range(50):
    """
    for x in xs:
        res = n(x)
        ypred.append(sum(res)/len(res))
    """
    # forward pass
    #ypred = [n(x) for x in xs]
    
    ypred = []
    for x in xs:
        print(n(x))
        res = ''.join(['0' if num.data < 0.5 else '1' for num in n(x)])
        ypred.append(Value(int(res, 2)))
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    # update
    for p in n.parameters():
        p.data += -0.01 * p.grad

    print(k, loss.data)
    
for pred, expctd in zip(ypred, ys):
    print(pred, expctd)
  



[Value(data=-0.2983793889433125), Value(data=0.3348828440302798), Value(data=0.4814224570525236), Value(data=-0.8737472303453331)]
[Value(data=-0.15843983421804764), Value(data=0.27004922060890335), Value(data=-0.30704461973736796), Value(data=0.10530848190753417)]
[Value(data=-0.26592786478341685), Value(data=0.10978689917100326), Value(data=-0.15084454611703882), Value(data=-0.13948853308632855)]
[Value(data=-0.6360459175264581), Value(data=0.5601264073366683), Value(data=0.17256776787094988), Value(data=-0.9582123327070724)]
[Value(data=-0.39789313429996054), Value(data=0.03786121818839695), Value(data=-0.023568464020016385), Value(data=-0.45614839625912834)]
0 297.0
[Value(data=-0.2983793889433125), Value(data=0.3348828440302798), Value(data=0.4814224570525236), Value(data=-0.8737472303453331)]
[Value(data=-0.15843983421804764), Value(data=0.27004922060890335), Value(data=-0.30704461973736796), Value(data=0.10530848190753417)]
[Value(data=-0.26592786478341685), Value(data=0.1097868

[Value(data=-0.2983793889433125), Value(data=0.3348828440302798), Value(data=0.4814224570525236), Value(data=-0.8737472303453331)]
[Value(data=-0.15843983421804764), Value(data=0.27004922060890335), Value(data=-0.30704461973736796), Value(data=0.10530848190753417)]
[Value(data=-0.26592786478341685), Value(data=0.10978689917100326), Value(data=-0.15084454611703882), Value(data=-0.13948853308632855)]
[Value(data=-0.6360459175264581), Value(data=0.5601264073366683), Value(data=0.17256776787094988), Value(data=-0.9582123327070724)]
[Value(data=-0.39789313429996054), Value(data=0.03786121818839695), Value(data=-0.023568464020016385), Value(data=-0.45614839625912834)]
34 297.0
[Value(data=-0.2983793889433125), Value(data=0.3348828440302798), Value(data=0.4814224570525236), Value(data=-0.8737472303453331)]
[Value(data=-0.15843983421804764), Value(data=0.27004922060890335), Value(data=-0.30704461973736796), Value(data=0.10530848190753417)]
[Value(data=-0.26592786478341685), Value(data=0.109786